In [2]:
from typing import List

import pandas as pd
from pathlib import Path
import os
import sys
from dotenv import load_dotenv, dotenv_values
from pydriller import Repository
import json
from multiprocessing import Pool
from itertools import chain

In [ ]:
load_dotenv()
ENV = dotenv_values(".env")
DATA_DIR = Path(ENV["DATA_DIR"])
DATA_DIR

In [4]:
df = pd.read_csv(DATA_DIR / 'interim' / 'interim_result.1.tsv', delimiter='\t')
df

,cveId,ghsaId,packageName,vulnIntroduced,latestVulnVersion,vulnFixed,fixCommitRepo,fixCommitHash
0,CVE-2019-0214,GHSA-jxgm-9f58-w4xp,org.apache.archiva:archiva,2.2.0,2.2.3,2.2.4,https://github.com/apache/archiva,796716d44183bd315dd20184a66b39ae533eb747
1,CVE-2013-7285,GHSA-f554-x222-wgf7,com.thoughtworks.xstream:xstream,0,1.4.5,1.4.6,https://github.com/x-stream/xstream,6344867dce6767af7d0fe34fb393271a6456672d
2,CVE-2019-0213,GHSA-cqcf-4g4h-rghf,org.apache.archiva:archiva,0,2.2.3,2.2.4,https://github.com/apache/archiva,a36035b49ba7d6514d6c386b51e1ad2512371b3d
3,CVE-2019-17513,GHSA-mvqp-q37c-wf9j,io.ratpack:ratpack-core,0,1.7.4,1.7.5,https://github.com/ratpack/ratpack,efb910d38a96494256f36675ef0e5061097dd77d
4,CVE-2017-15703,GHSA-xwx6-vmj4-5rv8,org.apache.nifi:nifi-framework-cluster-protocol,0,1.4.0,1.5.0,https://github.com/apache/nifi,9e2c7be7d3c6a380c5f61074d9a5a690b617c3dc
...,...,...,...,...,...,...,...,...
199,CVE-2020-9296,GHSA-wfj5-2mqr-7jvv,com.netflix.conductor:conductor-core,0,2.25.3,2.25.4,https://github.com/Netflix/conductor,375f8d8b708a801647c327e0848b30d34f804edd
200,CVE-2020-13692,GHSA-88cc-g835-76rp,org.postgresql:postgresql,0,42.2.12.jre7,42.2.13,https://github.com/pgjdbc/pgjdbc,14b62aca4764d496813f55a43d050b017e01eb65
201,CVE-2020-1758,GHSA-c597-f74m-jgc2,org.keycloak:keycloak-parent,0,9.0.3,10.0.0,https://github.com/keycloak/keycloak,97b565469058cacac6d2559b2cff37effcca2cfd
202,CVE-2018-21234,GHSA-jrg3-qq99-35g7,org.jodd:jodd-all,0,5.0.3,5.0.4,https://github.com/oblac/jodd,9bffc3913aeb8472c11bb543243004b4b4376f16


In [5]:
df_dict = df.to_dict(orient='records')

In [11]:
def get_new_records(r) -> List:
    new_records = list()
    repo_path = DATA_DIR / 'interim' / 'repositories'
    repo_path.mkdir(exist_ok=True, parents=True)

    try:
        repo = Repository(r["fixCommitRepo"], single=r["fixCommitHash"], clone_repo_to=repo_path)
    except Exception:
        print(f'error cloning {r["fixCommitRepo"]} ({r["fixCommitHash"]})')
        return []

    try:
        commit = list(repo.traverse_commits())[0]
    except Exception:
        print(f'error getting commit {r["fixCommitRepo"]} ({r["fixCommitHash"]})')
        return []

    # assert commit.hash == r["fixCommitHash"]
    for mf in commit.modified_files:
        try:
            if not mf.filename.endswith('.java') or mf.source_code_before is None:
                continue
            new_row = r.copy()
            new_row['commitHashFromGit'] = commit.hash
            new_row['modifiedFilePathBefore'] = mf.old_path
            new_row['modifiedFilePathAfter'] = mf.new_path
            new_row['modifiedFileSrcBefore'] = mf.source_code_before
            new_row['modifiedFileSrcAfter'] = mf.source_code
            new_row['diffParsedJson'] = json.dumps(mf.diff_parsed)
            new_row['nloc'] = mf.nloc
            new_row['changedMethods'] = list(map(lambda m: m.name, mf.changed_methods))
        
            new_records.append(new_row)

        except Exception:
            print(f'error parsing modified files {r["fixCommitRepo"]} ({r["fixCommitHash"]} ({mf.filename})')
            continue

    return new_records


with Pool(8) as p:
    new_records_list = p.map(get_new_records, df_dict)
    new_df = list(chain(*new_records_list))

pd.DataFrame(data=new_df)

error getting commit https://github.com/apache/poi (3a328aa220f6979f9805f658ae33244d153beaa7)
error getting commit http://svn.apache.org/repos/asf/xerces/java (787352)
error getting commit http://svn.apache.org/repos/asf/activemq (1209700)
error getting commit http://svn.apache.org/repos/asf/httpcomponents/httpclient (1074473)
error getting commit https://github.com/bcgit/bc-java (81b00861cd5711e85fe8dce2a0e119f684120255)


,cveId,ghsaId,packageName,vulnIntroduced,latestVulnVersion,vulnFixed,fixCommitRepo,fixCommitHash,commitHashFromGit,modifiedFilePathBefore,modifiedFilePathAfter,modifiedFileSrcBefore,modifiedFileSrcAfter,diffParsedJson,nloc,changedMethods
0,CVE-2019-0214,GHSA-jxgm-9f58-w4xp,org.apache.archiva:archiva,2.2.0,2.2.3,2.2.4,https://github.com/apache/archiva,796716d44183bd315dd20184a66b39ae533eb747,796716d44183bd315dd20184a66b39ae533eb747,archiva-modules/archiva-base/archiva-repositor...,archiva-modules/archiva-base/archiva-repositor...,package org.apache.archiva.admin.repository.ad...,package org.apache.archiva.admin.repository.ad...,"{""added"": [[24, ""import org.apache.archiva.adm...",396.0,"[DefaultArchivaAdministration::checkUrl, Defau..."
1,CVE-2019-0214,GHSA-jxgm-9f58-w4xp,org.apache.archiva:archiva,2.2.0,2.2.3,2.2.4,https://github.com/apache/archiva,796716d44183bd315dd20184a66b39ae533eb747,796716d44183bd315dd20184a66b39ae533eb747,archiva-modules/archiva-base/archiva-repositor...,archiva-modules/archiva-base/archiva-repositor...,package org.apache.archiva.admin.repository.ad...,package org.apache.archiva.admin.repository.ad...,"{""added"": [[225, "" newOrganisationI...",221.0,[ArchivaAdministrationTest::badOrganisationInf...
2,CVE-2019-0214,GHSA-jxgm-9f58-w4xp,org.apache.archiva:archiva,2.2.0,2.2.3,2.2.4,https://github.com/apache/archiva,796716d44183bd315dd20184a66b39ae533eb747,796716d44183bd315dd20184a66b39ae533eb747,archiva-modules/archiva-web/archiva-rest/archi...,archiva-modules/archiva-web/archiva-rest/archi...,package org.apache.archiva.rest.services;\n/*\...,package org.apache.archiva.rest.services;\n/*\...,"{""added"": [[23, ""import org.apache.archiva.adm...",382.0,[DefaultArchivaAdministrationService::setOrgan...
3,CVE-2019-0214,GHSA-jxgm-9f58-w4xp,org.apache.archiva:archiva,2.2.0,2.2.3,2.2.4,https://github.com/apache/archiva,796716d44183bd315dd20184a66b39ae533eb747,796716d44183bd315dd20184a66b39ae533eb747,archiva-modules/archiva-web/archiva-rest/archi...,archiva-modules/archiva-web/archiva-rest/archi...,package org.apache.archiva.rest.services;\n/*\...,package org.apache.archiva.rest.services;\n/*\...,"{""added"": [[26, ""import org.apache.archiva.res...",182.0,[ArchivaAdministrationServiceTest::badOrganiza...
4,CVE-2013-7285,GHSA-f554-x222-wgf7,com.thoughtworks.xstream:xstream,0,1.4.5,1.4.6,https://github.com/x-stream/xstream,6344867dce6767af7d0fe34fb393271a6456672d,6344867dce6767af7d0fe34fb393271a6456672d,xstream/src/java/com/thoughtworks/xstream/XStr...,xstream/src/java/com/thoughtworks/xstream/XStr...,"/*\n * Copyright (C) 2003, 2004, 2005, 2006 Jo...","/*\n * Copyright (C) 2003, 2004, 2005, 2006 Jo...","{""added"": [[3, "" * Copyright (C) 2006, 2007, 2...",1032.0,"[XStream::allowTypes, XStream::denyPermission,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,CVE-2018-21234,GHSA-jrg3-qq99-35g7,org.jodd:jodd-all,0,5.0.3,5.0.4,https://github.com/oblac/jodd,9bffc3913aeb8472c11bb543243004b4b4376f16,9bffc3913aeb8472c11bb543243004b4b4376f16,jodd-json/src/main/java/jodd/json/JsonParserBa...,jodd-json/src/main/java/jodd/json/JsonParserBa...,"// Copyright (c) 2003-present, Jodd Team (http...","// Copyright (c) 2003-present, Jodd Team (http...","{""added"": [[58, ""\tprotected List<String> clas...",87.0,[]
646,CVE-2018-21234,GHSA-jrg3-qq99-35g7,org.jodd:jodd-all,0,5.0.3,5.0.4,https://github.com/oblac/jodd,9bffc3913aeb8472c11bb543243004b4b4376f16,9bffc3913aeb8472c11bb543243004b4b4376f16,jodd-json/src/main/java/jodd/json/MapToBean.java,jodd-json/src/main/java/jodd/json/MapToBean.java,"// Copyright (c) 2003-present, Jodd Team (http...","// Copyright (c) 2003-present, Jodd Team (http...","{""added"": [[35, ""import jodd.util.Wildcard;""],...",163.0,"[MapToBean::map2bean, MapToBean::checkClassName]"
647,CVE-2018-21234,GHSA-jrg3-qq99-35g7,org.jodd:jodd-all,0,5.0.3,5.0.4,https://github.com/oblac/jodd,9bffc3913aeb8472c11bb543243004b4b4376f16,9bffc3913aeb8472c11bb543243004b4b4376f16,jodd-json/src/test/ja

last output:
error getting commit https://github.com/apache/poi (3a328aa220f6979f9805f658ae33244d153beaa7)
error getting commit http://svn.apache.org/repos/asf/xerces/java (787352)
error getting commit http://svn.apache.org/repos/asf/activemq (1209700)
error getting commit http://svn.apache.org/repos/asf/httpcomponents/httpclient (1074473)
error getting commit https://github.com/bcgit/bc-java (81b00861cd5711e85fe8dce2a0e119f684120255)

In [16]:
df2 = pd.DataFrame(data=new_df)
df2 = df2[~df2['modifiedFilePathAfter'].isnull()]
df2 = df2[~df2['modifiedFileSrcAfter'].isnull()]
df2

,cveId,ghsaId,packageName,vulnIntroduced,latestVulnVersion,vulnFixed,fixCommitRepo,fixCommitHash,commitHashFromGit,modifiedFilePathBefore,modifiedFilePathAfter,modifiedFileSrcBefore,modifiedFileSrcAfter,diffParsedJson,nloc,changedMethods
0,CVE-2019-0214,GHSA-jxgm-9f58-w4xp,org.apache.archiva:archiva,2.2.0,2.2.3,2.2.4,https://github.com/apache/archiva,796716d44183bd315dd20184a66b39ae533eb747,796716d44183bd315dd20184a66b39ae533eb747,archiva-modules/archiva-base/archiva-repositor...,archiva-modules/archiva-base/archiva-repositor...,package org.apache.archiva.admin.repository.ad...,package org.apache.archiva.admin.repository.ad...,"{""added"": [[24, ""import org.apache.archiva.adm...",396.0,"[DefaultArchivaAdministration::checkUrl, Defau..."
1,CVE-2019-0214,GHSA-jxgm-9f58-w4xp,org.apache.archiva:archiva,2.2.0,2.2.3,2.2.4,https://github.com/apache/archiva,796716d44183bd315dd20184a66b39ae533eb747,796716d44183bd315dd20184a66b39ae533eb747,archiva-modules/archiva-base/archiva-repositor...,archiva-modules/archiva-base/archiva-repositor...,package org.apache.archiva.admin.repository.ad...,package org.apache.archiva.admin.repository.ad...,"{""added"": [[225, "" newOrganisationI...",221.0,[ArchivaAdministrationTest::badOrganisationInf...
2,CVE-2019-0214,GHSA-jxgm-9f58-w4xp,org.apache.archiva:archiva,2.2.0,2.2.3,2.2.4,https://github.com/apache/archiva,796716d44183bd315dd20184a66b39ae533eb747,796716d44183bd315dd20184a66b39ae533eb747,archiva-modules/archiva-web/archiva-rest/archi...,archiva-modules/archiva-web/archiva-rest/archi...,package org.apache.archiva.rest.services;\n/*\...,package org.apache.archiva.rest.services;\n/*\...,"{""added"": [[23, ""import org.apache.archiva.adm...",382.0,[DefaultArchivaAdministrationService::setOrgan...
3,CVE-2019-0214,GHSA-jxgm-9f58-w4xp,org.apache.archiva:archiva,2.2.0,2.2.3,2.2.4,https://github.com/apache/archiva,796716d44183bd315dd20184a66b39ae533eb747,796716d44183bd315dd20184a66b39ae533eb747,archiva-modules/archiva-web/archiva-rest/archi...,archiva-modules/archiva-web/archiva-rest/archi...,package org.apache.archiva.rest.services;\n/*\...,package org.apache.archiva.rest.services;\n/*\...,"{""added"": [[26, ""import org.apache.archiva.res...",182.0,[ArchivaAdministrationServiceTest::badOrganiza...
4,CVE-2013-7285,GHSA-f554-x222-wgf7,com.thoughtworks.xstream:xstream,0,1.4.5,1.4.6,https://github.com/x-stream/xstream,6344867dce6767af7d0fe34fb393271a6456672d,6344867dce6767af7d0fe34fb393271a6456672d,xstream/src/java/com/thoughtworks/xstream/XStr...,xstream/src/java/com/thoughtworks/xstream/XStr...,"/*\n * Copyright (C) 2003, 2004, 2005, 2006 Jo...","/*\n * Copyright (C) 2003, 2004, 2005, 2006 Jo...","{""added"": [[3, "" * Copyright (C) 2006, 2007, 2...",1032.0,"[XStream::allowTypes, XStream::denyPermission,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,CVE-2018-21234,GHSA-jrg3-qq99-35g7,org.jodd:jodd-all,0,5.0.3,5.0.4,https://github.com/oblac/jodd,9bffc3913aeb8472c11bb543243004b4b4376f16,9bffc3913aeb8472c11bb543243004b4b4376f16,jodd-json/src/main/java/jodd/json/JsonParserBa...,jodd-json/src/main/java/jodd/json/JsonParserBa...,"// Copyright (c) 2003-present, Jodd Team (http...","// Copyright (c) 2003-present, Jodd Team (http...","{""added"": [[58, ""\tprotected List<String> clas...",87.0,[]
646,CVE-2018-21234,GHSA-jrg3-qq99-35g7,org.jodd:jodd-all,0,5.0.3,5.0.4,https://github.com/oblac/jodd,9bffc3913aeb8472c11bb543243004b4b4376f16,9bffc3913aeb8472c11bb543243004b4b4376f16,jodd-json/src/main/java/jodd/json/MapToBean.java,jodd-json/src/main/java/jodd/json/MapToBean.java,"// Copyright (c) 2003-present, Jodd Team (http...","// Copyright (c) 2003-present, Jodd Team (http...","{""added"": [[35, ""import jodd.util.Wildcard;""],...",163.0,"[MapToBean::map2bean, MapToBean::checkClassName]"
647,CVE-2018-21234,GHSA-jrg3-qq99-35g7,org.jodd:jodd-all,0,5.0.3,5.0.4,https://github.com/oblac/jodd,9bffc3913aeb8472c11bb543243004b4b4376f16,9bffc3913aeb8472c11bb543243004b4b4376f16,jodd-json/src/test/ja

For 11 records, the file has been deleted.

In [17]:
df2 = df2[~df2['modifiedFilePathBefore'].isnull()]
df2 = df2[~df2['modifiedFileSrcBefore'].isnull()]
df2

,cveId,ghsaId,packageName,vulnIntroduced,latestVulnVersion,vulnFixed,fixCommitRepo,fixCommitHash,commitHashFromGit,modifiedFilePathBefore,modifiedFilePathAfter,modifiedFileSrcBefore,modifiedFileSrcAfter,diffParsedJson,nloc,changedMethods
0,CVE-2019-0214,GHSA-jxgm-9f58-w4xp,org.apache.archiva:archiva,2.2.0,2.2.3,2.2.4,https://github.com/apache/archiva,796716d44183bd315dd20184a66b39ae533eb747,796716d44183bd315dd20184a66b39ae533eb747,archiva-modules/archiva-base/archiva-repositor...,archiva-modules/archiva-base/archiva-repositor...,package org.apache.archiva.admin.repository.ad...,package org.apache.archiva.admin.repository.ad...,"{""added"": [[24, ""import org.apache.archiva.adm...",396.0,"[DefaultArchivaAdministration::checkUrl, Defau..."
1,CVE-2019-0214,GHSA-jxgm-9f58-w4xp,org.apache.archiva:archiva,2.2.0,2.2.3,2.2.4,https://github.com/apache/archiva,796716d44183bd315dd20184a66b39ae533eb747,796716d44183bd315dd20184a66b39ae533eb747,archiva-modules/archiva-base/archiva-repositor...,archiva-modules/archiva-base/archiva-repositor...,package org.apache.archiva.admin.repository.ad...,package org.apache.archiva.admin.repository.ad...,"{""added"": [[225, "" newOrganisationI...",221.0,[ArchivaAdministrationTest::badOrganisationInf...
2,CVE-2019-0214,GHSA-jxgm-9f58-w4xp,org.apache.archiva:archiva,2.2.0,2.2.3,2.2.4,https://github.com/apache/archiva,796716d44183bd315dd20184a66b39ae533eb747,796716d44183bd315dd20184a66b39ae533eb747,archiva-modules/archiva-web/archiva-rest/archi...,archiva-modules/archiva-web/archiva-rest/archi...,package org.apache.archiva.rest.services;\n/*\...,package org.apache.archiva.rest.services;\n/*\...,"{""added"": [[23, ""import org.apache.archiva.adm...",382.0,[DefaultArchivaAdministrationService::setOrgan...
3,CVE-2019-0214,GHSA-jxgm-9f58-w4xp,org.apache.archiva:archiva,2.2.0,2.2.3,2.2.4,https://github.com/apache/archiva,796716d44183bd315dd20184a66b39ae533eb747,796716d44183bd315dd20184a66b39ae533eb747,archiva-modules/archiva-web/archiva-rest/archi...,archiva-modules/archiva-web/archiva-rest/archi...,package org.apache.archiva.rest.services;\n/*\...,package org.apache.archiva.rest.services;\n/*\...,"{""added"": [[26, ""import org.apache.archiva.res...",182.0,[ArchivaAdministrationServiceTest::badOrganiza...
4,CVE-2013-7285,GHSA-f554-x222-wgf7,com.thoughtworks.xstream:xstream,0,1.4.5,1.4.6,https://github.com/x-stream/xstream,6344867dce6767af7d0fe34fb393271a6456672d,6344867dce6767af7d0fe34fb393271a6456672d,xstream/src/java/com/thoughtworks/xstream/XStr...,xstream/src/java/com/thoughtworks/xstream/XStr...,"/*\n * Copyright (C) 2003, 2004, 2005, 2006 Jo...","/*\n * Copyright (C) 2003, 2004, 2005, 2006 Jo...","{""added"": [[3, "" * Copyright (C) 2006, 2007, 2...",1032.0,"[XStream::allowTypes, XStream::denyPermission,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,CVE-2018-21234,GHSA-jrg3-qq99-35g7,org.jodd:jodd-all,0,5.0.3,5.0.4,https://github.com/oblac/jodd,9bffc3913aeb8472c11bb543243004b4b4376f16,9bffc3913aeb8472c11bb543243004b4b4376f16,jodd-json/src/main/java/jodd/json/JsonParserBa...,jodd-json/src/main/java/jodd/json/JsonParserBa...,"// Copyright (c) 2003-present, Jodd Team (http...","// Copyright (c) 2003-present, Jodd Team (http...","{""added"": [[58, ""\tprotected List<String> clas...",87.0,[]
646,CVE-2018-21234,GHSA-jrg3-qq99-35g7,org.jodd:jodd-all,0,5.0.3,5.0.4,https://github.com/oblac/jodd,9bffc3913aeb8472c11bb543243004b4b4376f16,9bffc3913aeb8472c11bb543243004b4b4376f16,jodd-json/src/main/java/jodd/json/MapToBean.java,jodd-json/src/main/java/jodd/json/MapToBean.java,"// Copyright (c) 2003-present, Jodd Team (http...","// Copyright (c) 2003-present, Jodd Team (http...","{""added"": [[35, ""import jodd.util.Wildcard;""],...",163.0,"[MapToBean::map2bean, MapToBean::checkClassName]"
647,CVE-2018-21234,GHSA-jrg3-qq99-35g7,org.jodd:jodd-all,0,5.0.3,5.0.4,https://github.com/oblac/jodd,9bffc3913aeb8472c11bb543243004b4b4376f16,9bffc3913aeb8472c11bb543243004b4b4376f16,jodd-json/src/test/ja

For 0 records, new file has been introduced.
639 records have been saved

In [18]:
df2.to_json(DATA_DIR/'interim'/'changes_from_git.json', orient='records', indent=2)

In [15]:
df2.columns.tolist()

['cveId',
 'ghsaId',
 'packageName',
 'vulnIntroduced',
 'latestVulnVersion',
 'vulnFixed',
 'fixCommitRepo',
 'fixCommitHash',
 'commitHashFromGit',
 'modifiedFilePathBefore',
 'modifiedFilePathAfter',
 'modifiedFileSrcBefore',
 'modifiedFileSrcAfter',
 'diffParsedJson',
 'nloc',
 'changedMethods']